In [3]:
search_for = 128
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.017089366912841797
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 330
all cases: 866251507
type: [1, 1, 1, 1, 128] 128
cases of this type: 268435456
100000 43.92697931160257
200000 69.4449154608333
300000 68.22831672598943
400000 66.61443296734029
500000 67.27831907431026
600000 70.48693684657603
700000 67.10881191027129
800000 66.74612678155152
900000 66.56825776499866
1000000 66.0933423313849
1100000 19.57869727218349
1200000 67.69591795167636
1300000 65.80022029727765
1400000 65.07755780513658
1500000 65.48723755370179
1600000 69.0930308152337
1700000 66.73185293266661
1800000 65.70111783028203
1900000 66.66650363805822
2000000 66.47212162309985
2100000 52.07775462544601
2200000 35.50399974602148
2300000 63.23732062726982
2400000 63.26225299967801
2500000 62.27026922701592
2600000 63.20912690551329
2700000 65.67283937453428
2800000 63.22739015044368
2900000 61.76742939523586
3000000 61.6444788134377
3100

30100000 74.22688716077818
30200000 73.72932314759761
30300000 70.29997967474159
30400000 59.542466464832174
30500000 36.933762151170264
30600000 67.52152517732982
30700000 72.10171827004969
30800000 70.54373936466563
30900000 74.51576704016624
31000000 73.7865056042396
31100000 73.73410450367368
31200000 71.07836907881591
31300000 36.489355207423394
31400000 73.12111519269521
31500000 75.56072164543131
31600000 71.53826837930978
31700000 59.51710449900314
31800000 47.377571226781114
31900000 75.43063040526017
32000000 69.58522053378357
32100000 71.1605332525801
32200000 69.92200917364255
32300000 65.61228681386741
32400000 65.18552067335327
32500000 52.69646370685885
32600000 41.00433076377546
32700000 67.12701830765356
32800000 66.35474679449406
32900000 67.92076697193707
33000000 71.5681400260144
33100000 74.46233425751012
33200000 69.9854060721268
33300000 58.17558125087195
33400000 47.29187604053357
33500000 72.88755693122859
33600000 76.23196726206021
33700000 76.19575420793105
3

60500000 71.65836647173529
60600000 75.7633592250818
60700000 74.35648463812463
60800000 79.95424902198376
60900000 76.89006746405997
61000000 76.04312858912948
61100000 73.3234923271288
61200000 75.89844854381151
61300000 51.38684809418103
61400000 50.0635535676115
61500000 69.7838043152067
61600000 64.4590912727191
61700000 67.61662669432818
61800000 66.48091904154637
61900000 28.76058948954864
62000000 65.1978248167262
62100000 64.74945391103662
62200000 63.816310312108044
62300000 69.20986792177152
62400000 69.45135164366779
62500000 32.8587237139828
62600000 71.88715240227364
62700000 76.98646431075106
62800000 72.93780927356171
62900000 77.45871922290065
63000000 84.00241559878532
63100000 75.670763622373
63200000 75.57401429691353
63300000 74.88670674746375
63400000 59.97838038829651
63500000 43.96449913435978
63600000 68.98197508454594
63700000 72.61877739762973
63800000 76.45880634617708
63900000 69.81646997592735
64000000 27.72675800190961
64100000 65.60456770693727
64200000 

90900000 42.26343553268619
91000000 49.78226091767304
91100000 61.97855424647132
91200000 59.26503162045762
91300000 25.28826729503596
91400000 63.76263498216345
91500000 61.720684945766
91600000 35.41724779656078
91700000 56.54652269020401
91800000 59.32713319679929
91900000 62.12440055745665
92000000 68.7592402606348
92100000 66.89901264030712
92200000 66.44652735584907
92300000 66.2148566703418
92400000 71.65976724039382
92500000 71.32841840339745
92600000 65.96576669879784
92700000 67.31353856532681
92800000 68.11453537011585
92900000 64.27148497469906
93000000 45.8209517184733
93100000 43.6504349472983
93200000 66.56384036501358
93300000 61.692934330852964
93400000 26.29084513703782
93500000 63.63214524027967
93600000 61.82695330479913
93700000 29.765246133174863
93800000 66.18896343759434
93900000 64.24269481526154
94000000 65.52115987671218
94100000 73.58568391052512
94200000 68.32025165303803
94300000 73.03602607247724
94400000 70.81796611540656
94500000 73.50308416344473
94600

120400000 60.94770910604732
120500000 34.518137855577244
120600000 28.71902720771604
120700000 29.77257148137208
120800000 44.470445384428466
120900000 57.8809244612294
121000000 62.418044262608014
121100000 58.09996841953561
121200000 59.82172226034348
121300000 58.92823821870653
121400000 64.93165346225095
121500000 60.51245085075265
121600000 68.30149634386909
121700000 66.0031459979978
121800000 67.26770417585338
121900000 64.43432862564352
122000000 62.28835772129288
122100000 60.97277209462832
122200000 59.99326015227341
122300000 59.69425551438235
122400000 67.24069263688955
122500000 60.83610692085804
122600000 41.85410774113608
122700000 20.49652108655356
122800000 23.6574412411817
122900000 51.002011464711266
123000000 58.77286151096808
123100000 63.03200704355569
123200000 63.682330454683125
123300000 62.293959934761396
123400000 64.08055893970715
123500000 62.349721497715215
123600000 61.71511809224464
123700000 66.13421797297856
123800000 66.1304332372439
123900000 63.2158

...

best so far: 0
type: [1, 2, 16, 1, 4] 128
cases of this type: 131072
799800000 0.38570890347169506
799900000 0.45861781403938134
best so far: 0
type: [1, 2, 16, 2, 2] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 4, 1] 128
cases of this type: 32768
800000000 0.2549208613482548
best so far: 0
type: [1, 2, 32, 1, 2] 128
cases of this type: 32768
best so far: 0
type: [1, 2, 32, 2, 1] 128
cases of this type: 16384
best so far: 0
type: [1, 2, 64, 1, 1] 128
cases of this type: 8192
best so far: 0
type: [1, 4, 1, 1, 32] 128
cases of this type: 4194304
800100000 0.5313573852816873
800200000 1.3198808577875374
800300000 1.0631959052535747
800400000 1.2870302067403754
800500000 1.1308160390726387
800600000 1.1319586207817574
800700000 1.006214855182795
800800000 1.1083630663911879
800900000 1.2926981265647073
801000000 0.8617675669523372
801100000 0.7685657685152729
801200000 0.9255631109311138
801300000 1.005947607789243
801400000 0.9625577812670708
801500000 0.8885498878823604


813400000 1.0335800231785834
813500000 1.0986935073091635
813600000 0.8405523042911138
813700000 0.8927747548799594
813800000 1.0299795385582466
813900000 1.4902161985365179
814000000 0.9082886688843728
814100000 1.0078735007237447
814200000 1.2516377339685434
814300000 1.1626153944284439
814400000 1.0527713148955993
814500000 1.5111329460693115
814600000 1.242132607917893
814700000 1.2952709208375328
814800000 1.1689882135791303
814900000 1.1476737036290467
815000000 1.2323028312709345
815100000 1.0682626693195802
815200000 1.3088203397775215
815300000 0.8084972404631509
815400000 1.1391246283303673
815500000 1.2665428035419046
815600000 1.0702015783552377
815700000 1.2779538533175063
815800000 0.8847495566139639
815900000 0.9422207853865789
816000000 1.9513411717796756
816100000 0.9643704099401613
816200000 1.1152149812489411
816300000 0.8582886509963433
816400000 0.73298335982398
816500000 1.2126558296960128
816600000 0.7704701721394083
816700000 1.5942912452451288
816800000 1.32788

841100000 0.2577405529331181
841200000 0.27838921584654186
841300000 0.17992010488146545
841400000 0.24204010739398069
841500000 0.1773095380561643
841600000 0.22611855623304647
841700000 0.24662520274710856
841800000 0.24229712422206004
best so far: 0
type: [2, 1, 1, 8, 8] 128
cases of this type: 2097152
841900000 0.36285531284178496
842000000 0.21248743217910462
842100000 0.23594107702080605
842200000 0.26537282580165067
842300000 0.17507833989325233
842400000 0.20583144975266124
842500000 0.23615020293532613
842600000 0.1631886450830698
842700000 0.22587298790064456
842800000 0.1844836519176364
842900000 0.17655505766770577
843000000 0.3430231524776161
843100000 0.30670318898759286
843200000 0.20395155655806527
843300000 0.19592475829881034
843400000 0.2490948214241783
843500000 0.26891070761686764
843600000 0.1767552824735966
843700000 0.19506707570441364
843800000 0.260367455370511
843900000 0.1512480462578396
best so far: 0
type: [2, 1, 1, 16, 4] 128
cases of this type: 1048576
8

861300000 0.03830204498587515
best so far: 0
type: [2, 2, 2, 2, 8] 128
cases of this type: 262144
861400000 0.03930758007051746
861500000 0.032244441140498056
861600000 0.019188937532658712
best so far: 0
type: [2, 2, 2, 4, 4] 128
cases of this type: 131072
861700000 0.0290732331128478
best so far: 0
type: [2, 2, 2, 8, 2] 128
cases of this type: 65536
861800000 0.022810506743659576
best so far: 0
type: [2, 2, 2, 16, 1] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 1, 8] 128
cases of this type: 131072
861900000 0.022399939229492346
best so far: 0
type: [2, 2, 4, 2, 4] 128
cases of this type: 65536
862000000 0.021361300022204713
best so far: 0
type: [2, 2, 4, 4, 2] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 8, 1] 128
cases of this type: 16384
862100000 0.017853826489350532
best so far: 0
type: [2, 2, 8, 1, 4] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 8, 2, 2] 128
cases of this type: 16384
best so far: 0
type: [2, 2, 8, 4, 1] 128
cases of t

866100000 0.0007846320613318019
best so far: 0
type: [8, 1, 1, 2, 8] 128
cases of this type: 32768
best so far: 0
type: [8, 1, 1, 4, 4] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 1, 8, 2] 128
cases of this type: 8192
best so far: 0
type: [8, 1, 1, 16, 1] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 1, 8] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 2, 2, 4] 128
cases of this type: 8192
866200000 0.0002530151537537575
best so far: 0
type: [8, 1, 2, 4, 2] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 8, 1] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 1, 4] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 4, 2, 2] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 4, 1] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 1, 2] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 2, 1] 128
cases of this type: 512
best so far: 0
type: [8, 1, 16, 1, 1] 128
cases of this type: 256
best so f